In [ ]:
from tools import *
import matplotlib.pyplot as plt
import tools
import ms
import os
import seaborn as sns
import numpy as np
import torch
import re
from tqdm import tqdm
from importlib import reload
import bio_helper
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2
reload(bio_helper)

# plt.style.use(["ieee", 'no-latex'])


In [ ]:

def to_tensor(frag_msms):
    frag_msms = [i.reshape(1, -1) for i in frag_msms]
    frag_msms = np.concatenate(frag_msms, axis=0)
    frag_msms /= frag_msms.max()
    return torch.from_numpy(frag_msms)
# name, msms_data = read_msms(
#     "/data/prosit/test/fig3a/pick_msms.txt")


def read_ions(msms, name):
    i_index = name.index("Intensities")
    m_index = name.index("Matches")
    mass_index = name.index("Masses")
    return [(i[m_index].split(";"), i[i_index].split(";"), i[mass_index].split(";")) for i in msms]

def get_sa_all(run_model, data_nce_cand, frag_msms):
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    def perpare_data(seqs, nces, charges):
        seqs = torch.from_numpy(seqs)
        nces = torch.from_numpy(nces).unsqueeze(1)
        charges = torch.from_numpy(charges)
        data = {}
        data["sequence_integer"] = seqs.to(device)
        data["collision_energy_aligned_normed"] = nces.to(device)
        data['precursor_charge_onehot'] = charges.to(device)
        data['peptide_mask'] = ms.helper.create_mask(seqs).to(device)
        return data

    run_model = run_model.to(device)
    with torch.no_grad():
        data = perpare_data(*data_nce_cand)
        sass = []
        for b in range(0, len(frag_msms), 2048):
            d = {k: v[b:b+2048] for k, v in data.items()}
            pred = run_model(d)
            gt_frag = to_tensor(frag_msms[b:b+2048]).to(device)
            gt_frag = gt_frag/gt_frag.max()
            sas = helper.predict_sa(gt_frag, pred, d)
            # sas = helper.spectral_angle(gt_frag, pred)
            sass.append(sas)
        all_sa = torch.cat(sass, dim=0)
    return all_sa



In [ ]:
name, msms_data = tools.read_msms(
    f"/data/prosit/figs/fig235/trypsin/maxquant/combined/txt/msms.txt")
msms_data = tools.filter_msms(name, msms_data)


In [ ]:
from random import choices
msms_data = msms_data[:100000]

In [ ]:
msms_data.sort(key=lambda x: int(x[name.index("id")]))
m_r = loc_msms_in_raw(msms_data, f"/data/prosit/figs/fig235/trypsin/raw")
m_r = sorted(m_r, key=lambda x: int(x[0][name.index("id")]))
print(len(m_r))

In [ ]:
ids = [int(x[name.index("id")]) for x in msms_data]
print(len(ids), len(set(ids)))


In [ ]:
peptides = [i[0][name.index("Modified sequence")].replace("_", "") for i in m_r]

In [ ]:
scores = [float(m[0][name.index("Score")]) for m in m_r]
label = [0 if m[0][name.index("Reverse")] else 1 for m in m_r]

sns.histplot([s for s, l in zip(scores, label)
             if l == 1], binwidth=10, label="target", color="blue")
ax = sns.histplot([s for s, l in zip(scores, label) if l == 0],
                  binwidth=10, label="decoy")
ax.set(ylabel="#PSMs", title="Andromeda score")
plt.legend()


In [ ]:
from ms import model
import ms

# run_model = model.TransProBest()
# run_model.load_state_dict(torch.load(
#     "../checkpoints/frag/nce/best_nce_frag_TransPro-6-3-128-0.1-256-512-0.1-0.3.pth", map_location="cpu"))
# trans_model = run_model.eval()
# run_model = model.PrositFrag()
# run_model.load_state_dict(torch.load(
#     "../checkpoints/frag/nce/best_nce_frag_PrositFrag-512-0.1.pth", map_location="cpu"))
# trans_model = run_model.eval()


run_model = model.PrositFrag()
run_model.load_state_dict(torch.load(
    "../checkpoints/best/best_valid_irt_PrositFrag-1024.pth", map_location="cpu"))
prosit = run_model.eval()

# run_model = model.PrositFrag()
# run_model.load_state_dict(torch.load(
#     "../checkpoints/frag/best_nce_frag_PrositFrag-512.pth", map_location="cpu"))
# prosit_nce = run_model.eval()


In [ ]:
matched_ions_pre = generate_matched_ions(m_r)
m_i = [bio_helper.match_all(i, 'yb') for i in tqdm(matched_ions_pre)]
frag_msms = [bio_helper.reverse_annotation(*i[:4]) for i in tqdm(m_i)]


In [ ]:
matched_ions_rever = generate_matched_ions_delta(m_r)
m_i_rever = [bio_helper.match_all(i, 'yb') for i in tqdm(matched_ions_rever)]
frag_msms_rever = [bio_helper.reverse_annotation(*i[:4]) for i in tqdm(m_i_rever)]


In [ ]:
data_nce_cand = generate_from_mi(matched_ions_pre, name, nces=33)
data_nce_cand_rever = generate_from_mi(matched_ions_rever, name, nces=33)

trans_sa = get_sa_all(trans_model, data_nce_cand, frag_msms)
trans_sa = trans_sa.cpu().numpy()
trans_sa_rever = get_sa_all(trans_model, data_nce_cand_rever, frag_msms_rever)
trans_sa_rever = trans_sa_rever.cpu().numpy()

prosit_sa = get_sa_all(prosit, data_nce_cand, frag_msms)
prosit_sa = prosit_sa.cpu().numpy()
prosit_sa_rever = get_sa_all(prosit, data_nce_cand_rever, frag_msms_rever)
prosit_sa_rever = prosit_sa_rever.cpu().numpy()

# prosit_nce_sa = get_sa_all(prosit_nce, data_nce_cand, frag_msms)
# prosit_nce_sa = prosit_nce_sa.cpu().numpy()


In [ ]:
collects = []
for i in tqdm(range(10)):
    data_random = generate_from_msms_random(msms_data, name, nces=33, mutal=0.1)
    prosit_sa_random = get_sa_all(prosit, data_random, frag_msms)
    prosit_sa_random = prosit_sa_random.cpu().numpy()
    collects.append(prosit_sa_random.reshape(-1, 1))
collects = np.concatenate(collects, axis=1)
prosit_sa_random = np.mean(collects, axis=1)
# trans_sa_random = get_sa_all(trans_model, data_random, frag_msms)
# trans_sa_random = trans_sa_random.cpu().numpy()


In [ ]:
import pandas as pd

# prosit_sa_random[prosit_sa >= prosit_sa_random] = 0

joint_pd = pd.DataFrame({"sa": trans_sa, "and": trans_sa_rever,
                        "label": ['decoy' if m[0][name.index("Reverse")] else "target" for m in m_r]})

ax = sns.jointplot(x="and", y='sa', data=joint_pd, hue='label')
ax.set_axis_labels('Reverse SA', 'SA')
# plt.legend()


In [ ]:
c = (prosit_sa < prosit_sa_rever)
see = [1 if m[0][name.index("Reverse")] else 0 for i, m in enumerate(m_r) if c[i]]
print(np.sum(see), c.sum())

sns.histplot(prosit_sa[c])


In [ ]:
print(np.mean([1 if m[0][name.index("Reverse")]
      else 0 for i, m in enumerate(m_r)]))


In [ ]:
import pandas as pd

joint_pd = pd.DataFrame({"sa": trans_sa, "and": trans_sa - trans_sa_rever, "label": ['decoy' if m[0][name.index("Reverse")] else "target" for m in m_r]})

ax = sns.jointplot(x="and", y='sa', data=joint_pd, hue='label')
ax.set_axis_labels('Reverse SA', 'SA')

In [ ]:
sns.histplot([s.item() for s, l in zip(trans_sa, label)
             if l == 1 and s.item() >= 0], binwidth=0.02, label="target", color="blue")
ax = sns.histplot([s.item() for s, l in zip(trans_sa, label) if l == 0 and s.item() >= 0],
                  binwidth=0.02, label="decoy")
ax.set(ylabel="#PSMs", title="SA(Ours)")
plt.legend()


In [ ]:
index = [i for i, (s, l) in enumerate(zip(trans_sa, label))
         if l == 1]

index_ = [i for i, (s, l) in enumerate(zip(trans_sa, label))
          if l == 0]
ax = sns.jointplot(x=[scores[i] for i in index], y=[trans_sa[i].item()
                   for i in index], label="target", color="green", s=12)
ax.set_axis_labels('Andromeda score', 'SA')

x = [scores[i] for i in index_]
y = [trans_sa[i].item() for i in index_]
ax.ax_joint.scatter(x=x, y=y, s=10, color="darkred",label="decoy")
# ax2 = sns.jointplot(x=[scores[i] for i in index_], y=[trans_sa[i].item()
#                                                       for i in index_], label="decoy", ax=ax)
# ax.legend()


In [ ]:
sns.jointplot(x=trans_sa, y=prosit_sa)

In [ ]:
per_name, per_data = tools.read_msms(
    f"/data/prosit/figs/fig235/trypsin/percolator_up/prosit_combined_weights.tab")


In [ ]:
len(per_data)

In [ ]:
labels = np.array([1 if int(i[per_name.index("Label")]) > 0 else 0 for i in per_data])
scores = np.array([float(i[per_name.index("andromeda")]) for i in per_data])
sa = np.array([float(i[per_name.index("spectral_angle")]) for i in per_data])


In [ ]:
sns.histplot([s for s, l in zip(sa, labels)
             if l == 1], binwidth=0.02, label="target", color="blue")
ax = sns.histplot([s for s, l in zip(sa, labels) if l == 0],
                  binwidth=0.02, label="decoy")
ax.set(ylabel="#PSMs", title="Andromeda score")
plt.legend()


In [ ]:
index = [i for i, (s, l) in enumerate(zip(sa, labels))
         if l == 1 and s.item() >= 0]

index_ = [i for i, (s, l) in enumerate(zip(sa, labels))
          if l == 0 and s.item() >= 0]
ax = sns.jointplot(x=[scores[i] for i in index], y=[sa[i]
                   for i in index], label="target", color="green", s=12)
ax.set_axis_labels('Andromeda score', 'SA')

x = [scores[i] for i in index_]
y = [sa[i] for i in index_]
ax.ax_joint.scatter(x=x, y=y, s=5, color="darkred", label="decoy")
# ax2 = sns.jointplot(x=[scores[i] for i in index_], y=[trans_sa[i].item()
#                                                       for i in index_], label="decoy", ax=ax)
# ax.legend()


In [ ]:
sns.histplot([s for s, l in zip(sa, labels)
             if l == 1], binwidth=0.02, label="target", color="blue")
ax = sns.histplot([s for s, l in zip(sa, labels) if l == 0],
                  binwidth=0.02, label="decoy")
ax.set(ylabel="#PSMs", title="SA(Ours)")
plt.legend()


In [ ]:
m = per_data[0]
print(sa[0], scores[0], labels[0], per_name)

In [ ]:
from collections import defaultdict
per_p_index = per_name.index("Peptide")
score_p_index = per_name.index("andromeda")
sa_p_index = per_name.index("spectral_angle")
label_p_index = per_name.index("Label")
mass_p_index = per_name.index("delta_score")
per_index = defaultdict(list)

for p in per_data:
    assert p[per_p_index][:2] == "_."
    assert p[per_p_index][-2:] == "._"
    p_p = p[per_p_index][2:-2]
    p_s = float(p[score_p_index])
    p_sa = float(p[sa_p_index])
    per_index[p_p].append((p_s, p_sa, int(p[label_p_index]), float(p[mass_p_index])))


In [ ]:

matches = []
for m, s in tqdm(zip(m_r, prosit_sa)):
    msms = m[0]
    pep = msms[name.index("Modified sequence")].strip("_")
    scores = float(msms[name.index("Score")])
    l1 = -1 if msms[name.index("Reverse")] else 1
    mass_m = float(msms[name.index("Delta score")])
    cand = per_index[pep]
    if len(cand):
        for c in cand:
            if abs(scores - c[0]) < 1e-8 and abs(mass_m - c[3]) < 1e-8 and l1==c[2]:
                matches.append((s.item(), c[1], c[2]))
                break

In [ ]:
len(matches)

In [ ]:
# plt.scatter([i[0] for i in matches if i[2] < 0], [i[1]
#             for i in matches if i[2] < 0], s=2)

ax = sns.jointplot(x=[i[0] for i in matches if i[2] <0], y=[i[1] for i in matches if i[2] <0], label="decoy", color="green", s=12)
ax.set_axis_labels("Our SA", "Percolator SA")


In [ ]:
ax = sns.jointplot(x=[i[0] for i in matches if i[2] > 0], y=[i[1]
                   for i in matches if i[2] > 0], label="target", color="green", s=12)
ax.set_axis_labels("Our SA", "Percolator SA")


In [ ]:
per_name

In [ ]:
[i[per_name.index('Mass')] for i in per_data]


In [ ]:
[i[name.index('Mass')] for i in msms_data]